## Read In Data

In [27]:
# !pip install transformers

In [28]:
import json
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from google.colab import drive
from transformers import BertTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

punctuation
stop word
limit to first X words
append tilile
lematize

In [9]:
# Mount Google Drive
drive.mount('/content/gdrive')

# Define the file path
path = "/content/gdrive/MyDrive/Advanced ML/articles.json"

# Read the JSON file
with open(path, 'r') as file:
    data = json.load(file)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
# Print a sample of data
num_rows= 2
for i in range(num_rows):
    print(data[i])



{'title': '\n\t\tTrump DC hotel falls millions short of projected revenue in first months\t', 'article_text': 'President Trump’s Washington, D.C., hotel made at least $2 million less than projected in its first two months of operation, according to House Democrats citing documents obtained from the General Services Administration (GSA).Democrats on the House Oversight and Transportation committees revealed the information in a letter sent Monday to GSA Administrator Timothy Horne, in which they asked how the agency planned to handle the alleged breach of the lease on the hotel housed in the Old Post Office, just east of the White House on Pennsylvania Avenue.A provision in the lease, signed years before Trump began his presidential campaign, prevents any federal elected official from being included on or benefiting from the lease. The Democrats claim in their letter that Trump’s ascendance to the presidency on Friday is a breach of the lease.The documents Democrats obtained from the GS

In [12]:
#development sample
data_sample = data[0:30]

# Specify the file path in Google Drive
file_path = "/content/gdrive/MyDrive/Advanced ML/30_sample_articles.json"

# Write the JSON data to the file
with open(file_path, "w") as fp:
    json.dump(data_sample, fp)

In [13]:
for article in data_sample:
    for key in article:
        print(key)
    break

title
article_text
images
uuid
parent


In [14]:
data[300].keys()

dict_keys(['title', 'article_text', 'images', 'uuid', 'parent'])

Looks like we could either grab sourse from image url main or imag url social

In [15]:
for sample in data_sample:
    #print(sample["images"])
    for url_item in sample["images"]:
        print(url_item["image_type"], url_item["url"])
    break



main https://i0.wp.com/thehill.com/wp-content/uploads/sites/2/2017/01/trumphotel618289268.jpg?w=2000&ssl=1
social https://thehill.com/wp-content/uploads/sites/2/2017/01/trumphotel618289268.jpg?w=1280


# Experimenting with preprocessing

In [ ]:
# isolate the article text
#text_only = []
#for article in data_sample:
#    text_only.append(article["article_text"])


In [16]:
json_string = json.dumps(data_sample)
df = pd.read_json(json_string)
df.head()

,title,article_text,images,uuid,parent
0,\n\t\tTrump DC hotel falls millions short of p...,"President Trump’s Washington, D.C., hotel made...",[{'url': 'https://i0.wp.com/thehill.com/wp-con...,73ff4208-406e-11ee-a96e-33dec8f414a2,73ff4208-406e-11ee-a96e-33dec8f414a2
1,\n\t\tOvernight Tech: Microsoft wants end to f...,MICROSOFT CALLS OUT GOVERNMENT HACKING PRACTIC...,[{'url': 'https://i0.wp.com/thehill.com/wp-con...,77a0c19b-406e-11ee-a96e-33dec8f414a2,77a0c19b-406e-11ee-a96e-33dec8f414a2
2,\n\t\tAnother US motorcycle company considerin...,Minnesota-based Polaris acknowledged on Friday...,[{'url': 'https://thehill.com/wp-content/uploa...,8cfe90c9-406e-11ee-a96e-33dec8f414a2,8cfe90c9-406e-11ee-a96e-33dec8f414a2
3,\n\t\tTrump: ‘I don’t need a summit’ with Mexi...,President Trump said he does not need to hold ...,[{'url': 'https://i0.wp.com/thehill.com/wp-con...,9cdbdae6-406e-11ee-a96e-33dec8f414a2,9cdbdae6-406e-11ee-a96e-33dec8f414a2
4,\n\t\tUS calls emergency meeting of UN nuclear...,The United States called an emergency meeting ...,[{'url': 'https://thehill.com/wp-content/uploa...,a3a5f190-406e-11ee-a96e-33dec8f414a2,a3a5f190-406e-11ee-a96e-33dec8f414a2


# Preprocessing

In [17]:
#concate text and title
df["title_text"] = df["title"] + " " +  df["article_text"]

In [18]:

#lowercase text
df['title_text'] = df['title_text'].str.lower()

# remove some unicode, for tabs and newline and return.
# are there others?
df['title_text'] = df['title_text'].apply(lambda x: re.sub(r'[\n\t\r]', '', x))

# remove punctuation, replace with space
df['title_text'] = df['title_text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

In [19]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'my', 'while', "that'll", 'shan', 't', 'that', 'shouldn', 'herself', 'by', 'll', 'are', 'above', 'ain', 'up', 'just', "mustn't", 'he', 'or', 'has', 'if', "needn't", 'other', 'about', 'hadn', 'who', 'their', 'than', 'yourselves', "didn't", 'should', "wasn't", 'in', 'wouldn', 'too', 'each', 'own', "it's", 'against', 'i', 'not', 's', 'until', 'isn', 'as', 'o', 'me', 'have', 'ourselves', 'didn', 'again', 'don', 'does', 'haven', "wouldn't", "you've", 'and', 'ours', 're', 'a', 'was', "couldn't", 'to', 'on', 'during', 'needn', "shan't", "she's", 'being', 'with', 'all', 'over', 'these', 'we', 'an', "don't", 'himself', 'but', 'won', 'been', 'yours', 'can', 'what', "you'll", 'whom', 'before', "shouldn't", 'some', 'here', 'them', 'where', 'now', "weren't", "hadn't", 'had', 'further', 'into', 'do', "won't", 'were', 'doing', 'down', 'off', 'from', 'when', 've', 'both', 'am', "you'd", 'him', 'which', 'nor', 'they', "should've", 'there', 'hasn', "you're", 'this', 'm', 'having', 'it', 'the', 'will', 

In [20]:
len(stop_words)

179

In [21]:

def remove_stopwords_and_lemmatize(text):

    word_tokens = word_tokenize(text)
    removed_stopword = [word for word in word_tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(token) for token in removed_stopword]

    return ' '.join(lemmatized_text[:50])

df['title_text'] = df['title_text'].apply(remove_stopwords_and_lemmatize)


In [22]:
# remove extra spaces
df['title_text_clean'] = df['title_text'].str.replace(r'\s+', ' ', regex=True)

In [23]:
df['title_text_clean'].head()
# US--> U?

0    trump dc hotel fall million short projected re...
1    overnight tech microsoft want end fed secret h...
2    another u motorcycle company considering movin...
3    trump need summit mexico el salvador guatemala...
4    u call emergency meeting un nuclear watchdog g...
Name: title_text_clean, dtype: object

In [25]:
#cleaned sample

#selected_columns = df[['title', 'text', 'title_text_clean']]
df[['title', 'article_text', 'title_text_clean']].to_csv("/content/gdrive/MyDrive/Advanced ML/30_sample_articles_cleaned.csv", index=False)

## Create embeddings

In [31]:
df['title_text_clean'][1]

'overnight tech microsoft want end fed secret hacking tool latest ransomware attack john oliver call civil tone net neutrality fight house vote cyber bill microsoft call government hacking practice heel widespread ransomware attack may used leaked national security agency hacking method microsoft calling government cease stockpiling secret mean bypassing software'

In [39]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text
text = 'overnight tech microsoft want end fed secret hacking tool latest ransomware attack john oliver call civil tone net neutrality fight house vote cyber bill microsoft call government hacking practice heel widespread ransomware attack may used leaked national security agency hacking method microsoft calling government cease stockpiling secret mean bypassing software'

encoding = tokenizer.encode_plus(text, add_special_tokens = True, truncation = True, return_tensors = "pt")

# Print the tokens
print("Original Text:", text)
print("Tokenized Text:", encoding)

Original Text: overnight tech microsoft want end fed secret hacking tool latest ransomware attack john oliver call civil tone net neutrality fight house vote cyber bill microsoft call government hacking practice heel widespread ransomware attack may used leaked national security agency hacking method microsoft calling government cease stockpiling secret mean bypassing software
Tokenized Text: {'input_ids': tensor([[  101, 11585,  6627,  7513,  2215,  2203,  7349,  3595, 23707,  6994,
          6745, 16540,  8059,  2886,  2198,  6291,  2655,  2942,  4309,  5658,
         21083,  2954,  2160,  3789, 16941,  3021,  7513,  2655,  2231, 23707,
          3218, 12073,  6923, 16540,  8059,  2886,  2089,  2109, 15748,  2120,
          3036,  4034, 23707,  4118,  7513,  4214,  2231, 13236,  4518,  8197,
          2989,  3595,  2812, 11826,  2075,  4007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0,